In [10]:
# Comment out these lines
import sys
sys.path.insert(0, 'C:\\Users\\masch\\Quantum Computing\\QComp\\pgmpy')

# Imports
import cmath
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.inference import BeliefPropagation

p1 = 0.1
p2 = 0.3

corr_bit_flip = BayesianNetwork([('q0m0','q0m1'),('q1m0','q1m1'),('rv1','rv2'),('rv1','q0m1'),('rv2','q1m1')])
cpd_q0m0 = TabularCPD(variable='q0m0',variable_card=2,values=[[1],[0]])
cpd_q1m0 = TabularCPD(variable='q1m0',variable_card=2,values=[[1],[0]])
cpd_rv1 = TabularCPD(variable='rv1',variable_card=2,values=[[np.sqrt(1-p1)],[np.sqrt(p1)]])
cpd_rv2 = TabularCPD(variable='rv2',variable_card=2,values=[[np.sqrt(1-p1),np.sqrt(1-p2)],[np.sqrt(p1),np.sqrt(p2)]],evidence=['rv1'],evidence_card=[2])
cpd_q0m1 = TabularCPD(variable='q0m1',variable_card=2,values=[[1,0,0,1],[0,1,1,0]],evidence=['q0m0','rv1'],evidence_card=[2,2])
cpd_q1m1 = TabularCPD(variable='q1m1',variable_card=2,values=[[1,0,0,1],[0,1,1,0]],evidence=['q1m0','rv2'],evidence_card=[2,2])

corr_bit_flip.add_cpds(cpd_q0m0,cpd_q1m0,cpd_rv1,cpd_rv2,cpd_q0m1,cpd_q1m1)
corr_infer = VariableElimination(corr_bit_flip)
corr_query = corr_infer.query(['rv1','rv2','q0m1','q1m1'])
print(corr_query)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------+---------+--------+---------+--------------------------+
| rv2    | q1m1    | rv1    | q0m1    |   phi(rv2,q1m1,rv1,q0m1) |
+========+=========+========+=========+==========================+
| rv2(0) | q1m1(0) | rv1(0) | q0m1(0) |           0.9000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv2(0) | q1m1(0) | rv1(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv2(0) | q1m1(0) | rv1(1) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv2(0) | q1m1(0) | rv1(1) | q0m1(1) |           0.2646+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv2(0) | q1m1(1) | rv1(0) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv2(0) | q1m1(1) | rv1(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+-----------------------

In [11]:
def cpd_2_dm(obj,rvs,var):
    numQubits = len(var)
    numRVs = len(rvs)
    varOrder = obj.variables
    numVars = len(varOrder)
    qubitOrdering = []
    rvsOrdering = []
    
    for i in range(numQubits):
        v = var[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        qubitOrdering.append(2**(numVars - j - 1))
        
    for i in range(numRVs):
        v = rvs[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        rvsOrdering.append(2**(numVars - j - 1))

    vals = (obj.values).flatten()
    dm = np.zeros((2**numQubits,2**numQubits),dtype="complex_")
    numEvents = 2**numRVs
    numPermutations = 2**numQubits
    
    for i in range(numEvents):
        val1 = 0
        for j in range(numRVs):
            val1 += ((i//(2**j))%2)*rvsOrdering[numRVs - j - 1]
        arr1 = np.zeros((numPermutations,1),dtype="complex_")
        arr2 = np.zeros((1,numPermutations),dtype="complex_")
        for j in range(numPermutations):
            val2 = val1
            for k in range(numQubits):
                val2 += ((j//(2**k))%2)*qubitOrdering[numQubits - k - 1]
            arr1[j][0] = vals[val2]
            arr2[0][j] = np.conj(vals[val2])
        dm += np.matmul(arr1,arr2)
        
    return dm

In [12]:
X = cpd_2_dm(corr_query,['rv1','rv2'],['q0m1', 'q1m1']).round(4)
print(X)

[[0.81+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.09+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.07+0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.  +0.j 0.03+0.j]]
